## Diving Deeper: Alternatives for importing models

In [Getting Started][colab-getting-started], we imported a model from Hugging Face. Here, we will demonstrate what it looks like to import models from [GitHub][github], [PyPI][pypi], and jatic_toolbox.

[colab-getting-started]: https://colab.research.google.com/github/twosixlabs/armory-library/blob/master/docs/getting_started.ipynb

[pypi]: https://pypi.org
[github]: https://github.com


#### GitHub

The following is an example of how to import a model from GitHub, after having
cloned the relevant repository:
```
git clone 'https://github.com/Lornatang/SRGAN-PyTorch'
```

```
import sys
sys.path.insert(0,'/SRGAN-PyTorch')
from SRGAN-PyTorch import model as pytorch_new_model

SRRmodel = pytorch_new_model.SRResNet()

model = JaticImageClassificationModel(
    SRRmodel
)
```

Here, we add the GitHub folder to the system path and import the mode. Then,
`JaticImageClassificationModel` is a wrapper to make the model compatible with
Armory.

#### PyPI

The following is an example of loading a model from the [EfficientNet Lite][eff-lite]
PyTorch library:

[eff-lite]: https://pypi.org/project/efficientnet-lite-pytorch/

In [ ]:
from efficientnet_lite_pytorch import EfficientNet
from efficientnet_lite0_pytorch_model import EfficientnetLite0ModelFile

from charmory.model.image_classification import JaticImageClassificationModel

weights_path = EfficientnetLite0ModelFile.get_model_file_path()

lite0_model = EfficientNet.from_pretrained('efficientnet-lite0', weights_path = weights_path )

model = JaticImageClassificationModel(
    lite0_model
)

Here, we import the library and create a model from the case with a weights path
added.

#### jatic_toolbox

The following is an example of loading a [TorchVision][torchvision] model from the jatic_toolbox:

[torchvision]: https://pytorch.org/vision/stable/index.html

In [ ]:
from jatic_toolbox import load_model

from charmory.track import track_params

model = track_params(load_model)(
        provider="torchvision",
        model_name="resnet34",
        task="image-classification",
    )

`load_model` takes the self-explanatory parameters `provider`, `model_name`, and `task`, and in this case, we choose an image classification model named [resnet34][resnet34]. It can be accessed directly via the PyTorch Hub without having TorchVision installed.

In each case, we use the `PyTorchClassifier` wrapper to make the model
compatible with the ART library. Note that this is specific to image
classification models written within the PyTorch framework. The parameters can
be adjusted as needed and are more thoroughly described in [Getting Started][colab-getting-started]. `track_initial_params` is used so that these parameters
are also tracked in MLflow.

[resnet34]: https://pytorch.org/vision/main/models/generated/torchvision.models.resnet34.html
[colab-getting-started]: https://colab.research.google.com/github/twosixlabs/armory-library/blob/master/docs/getting_started.ipynb

In [ ]:
from art.estimators.classification import PyTorchClassifier

from charmory.track import track_init_params
import torch

classifier = track_init_params(PyTorchClassifier)(
    model,
    loss=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=0.003),
    input_shape=(3, 224, 224),
    channels_first=True,
    nb_classes=10,
    clip_values=(-1, 1),
)